In [26]:
import joblib
import tensorflow as tf
import numpy as np

In [2]:
%%time
featurizer = joblib.load('featurizer.pkl')
model = tf.keras.models.load_model('model.h5')

CPU times: user 8.03 s, sys: 1.23 s, total: 9.26 s
Wall time: 10.7 s


In [3]:
%%time
testbib = '1 . Egbor M , Ansari T , Morris N , Green CJ , Sibbons PD . Morphometric placental villous and vascular abnormalities in early- and late - onset pre - eclampsia with and without fetal growth restriction . BJOG . 2006;113:580 - 9 . \n 2 . Macara L , Kingdom JC , Kaufmann P , Kohnen G , Hair J , More IA , Lyall F , Greer IA . Structural analysis of placental terminal villi from growth - restricted pregnancies with abnormal umbilical artery Doppler waveforms . Placenta . 1996;17:37 - 48 . \n 3 . Salafia CM , Ernst LM , Pezzullo JC , Wolf EJ , Rosenkrantz TS , Vintzileos AM . The very low birthweight infant : maternal complications leading to preterm birth , placental lesions , and intrauterine growth . Am J Perinatol . 1995;12:106 - 10 . \n 4 . Salgado SS , Salgado MKR . Structural changes in pre - eclamptic and eclamptic placentas - an ultrastructural study . J Coll Physicians Surg Pak . 2011;21:482 - 6 . \n 5 . Higgins M , Felle P , Mooney EE , Bannigan J , McAuliffe FM . Stereology of the placenta in type 1 and type 2 diabetes . Placenta . 2011;32:564 - 9 . \n 6 . Wislocki GB , Dempsey EW . Electron Microscopy of the human placenta . Anat Rec . 1955;123:133 - 67 . \n 7 . Terzakis J , Rhodin AG . Ultrastructure of Human Term Placenta . Anat Rec . 1961;139:279 . \n 8 . Leiser R , Luckhardt M , Kaufmann P , Winterhager E , Bruns U. The fetal vascularisation of term human placental villi . I. Peripheral stem villi . Anat Embryol ( Berl ) . 1985;173:71 - 80 . \n 9 . Leiser R , Krebs C , Ebert B , Dantzer V. Placental vascular corrosion cast studies : A comparison between ruminants and humans . Microsc Res Tech . 1997;38:76 - 87 . \n 10 . Langheinrich AC , Wienhard J , Vormann S , Hau B , Bohle RM , Zygmunt M. Analysis of the fetal placental vascular tree by X - ray micro - computed tomography . Placenta . 2004;25:95 - 100 . \n 11 . Hata T , Tanaka H , Noguchi J , Hata K. Three - dimensional ultrasound evaluation of the placenta . Placenta . 2011;32:105 - 15 . \n 12 . Jirkovská M , Kubínová L , Janáček J , Kaláb J. 3-D study of vessels in peripheral placental villi . Image Anal Stereol . 2007;26:165 - 168 . \n 13 . Demir R , Kosanke G , Kohnen G , Kertschanska S , Kaufmann P. Classification of human placental stem villi : review of structural and functional aspects . Microsc Res Tech . 1997;38:29 - 41 . \n 14 . Castellucci M , Scheper M , Scheffen I , Celona A , Kaufmann P. The development of the human placental villous tree . Anat Embryol ( Berl ) . 1990;181:117 - 28 . \n 15 . Roberts N , Magee D , Song Y , Brabazon K , Shires M , Crellin D , Orsi NM , Quirke R , Quirke P , Treanor D. Toward routine use of 3D histopathology as a research tool . Am J Pathol . 2012;180:1835 - 42 . \n 16 . Magee D , Treanor D , Quirke P. A New Image Registration algorithm with application to 3D Histopathology . In : Microscopic Image Analysis with Applications in Biology ( 2008 ) . \n'

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 14.5 µs


/home/joshib/.cache/pypoetry/virtualenvs/citation-splitter-serve-nqNJR9hR-py3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
%%time
feat_dict = featurizer.transform([testbib])

100%|██████████| 1/1 [00:00<00:00, 5511.57it/s]


CPU times: user 767 ms, sys: 10.4 ms, total: 778 ms
Wall time: 774 ms


In [5]:
%%time
feat_aux = np.concatenate((feat_dict['spacy_num_feats'], feat_dict['custom_feats']), axis=-1)
feat_lstm = feat_dict['lstm_feats']

CPU times: user 538 µs, sys: 105 µs, total: 643 µs
Wall time: 315 µs


In [6]:
%%time
probs = model.predict([feat_lstm, feat_aux])
probs = probs.reshape(probs.shape[0], probs.shape[1])
preds = np.where(probs > 0.5, 1, 0)

CPU times: user 1.45 s, sys: 285 ms, total: 1.73 s
Wall time: 1.71 s


In [7]:
%%time
take_mean = True
probs_win = []
for ind in range(len(probs)):
    #print(ind)
    probs_win.append(probs[ind][0:len(feat_dict['sentences_window'][ind])])

probs_merged = []
window = featurizer.feat_config['window']
step = featurizer.feat_config['step']
gind = 0
#for bigind in range(len(sentences_test)):
merged_probs = []
seqlen =  len(testbib.split(" "))
iter = 0
for smallind in range(gind, len(probs_win)):
    if len(merged_probs) < seqlen:
        
        if iter == 0:
            merged_probs += probs_win[gind].tolist()
            iter +=1
            gind += 1
            
        else:
            merged_probs[-(window-step):] = np.mean([np.array(merged_probs[-(window-step):]), probs_win[gind][0:(window-step)]], axis=0).tolist()
            merged_probs += probs_win[gind][(window-step):].tolist()
            gind += 1     
    else:
        break

CPU times: user 3.74 ms, sys: 726 µs, total: 4.47 ms
Wall time: 2.91 ms


In [8]:
merged_preds = [int(t>0.5) for t in merged_probs]

In [9]:
output = [" ".join(i.tolist()) for i in np.split(testbib.split(" "), np.where(np.array(merged_preds) == 1)[0][1:])]

In [10]:
output

['1 . Egbor M , Ansari T , Morris N , Green CJ , Sibbons PD . Morphometric placental villous and vascular abnormalities in early- and late - onset pre - eclampsia with and without fetal growth restriction . BJOG . 2006;113:580 - 9 . \n',
 '2 . Macara L , Kingdom JC , Kaufmann P , Kohnen G , Hair J , More IA , Lyall F , Greer IA . Structural analysis of placental terminal villi from growth - restricted pregnancies with abnormal umbilical artery Doppler waveforms . Placenta . 1996;17:37 - 48 . \n',
 '3 . Salafia CM , Ernst LM , Pezzullo JC , Wolf EJ , Rosenkrantz TS , Vintzileos AM . The very low birthweight infant : maternal complications leading to preterm birth , placental lesions , and intrauterine growth . Am J Perinatol . 1995;12:106 - 10 . \n',
 '4 . Salgado SS , Salgado MKR . Structural changes in pre - eclamptic and eclamptic placentas - an ultrastructural study . J Coll Physicians Surg Pak . 2011;21:482 - 6 . \n',
 '5 . Higgins M , Felle P , Mooney EE , Bannigan J , McAuliffe F